_Binary Classification of Skin Lesions_

In this lab assignment, implement the ResNet-152 model, a deep convolutional neural network, for classifying skin lesions into benign or malignant (binary classification). 
1. Use the dataset provided, write a custom dataset class to load the dataset properly.  
1. Do not use prebuilt ResNet architecture that comes with tensorflow/pytorch frameworks, instead create architecture using basic layers and functions from the framework like Conv2d, Linear, Relu, Pooling and others required.
1. Print model parameters and architecture to verify it matches the standard one.
1. Experiment with different values of hyperparameters, describe the observation at each step of experimentation. (important - must be documented in the submission file)
1. After training the model, evaluate the model’s performance on the test dataset. 
1. Calculate classification metrics such as accuracy, precision, recall, and F1-score.
1. Finally, summarize your observations/learnings after completing this assignment task.

Try appropriate batch size to train your model using Colab, Kaggle platform.


In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

2025-03-05 04:53:23.398157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741150403.420114    5802 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741150403.426675    5802 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 04:53:23.460522: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def residual_block(x,filters,stride=1,downsample=False):
    x_skip=x

    #First Conv layer
    x=tf.keras.layers.Conv2D(filters,kernel_size=1,strides=1,padding='valid')(x)
    x=tf.keras.layers.BatchNormalization()(x)
    x=tf.keras.layers.Activation('relu')(x)

    #Second Conv Layer
    x=tf.keras.layers.Conv2D(filters,kernel_size=3,strides=stride,padding='same')(x)
    x=tf.keras.layers.BatchNormalization()(x)
    x=tf.keras.layers.Activation('relu')(x)
    
    #Third Conv Layer
    x=tf.keras.layers.Conv2D(filters*4,kernel_size=1,strides=1,padding='valid')(x)
    x=tf.keras.layers.BatchNormalization()(x)

    #Apply downsampling if required
    if downsample:
        x_skip=tf.keras.layers.Conv2D(filters*4,kernel_size=1,strides=stride,padding='valid')(x_skip)
        x_skip=tf.keras.layers.BatchNormalization()(x_skip)
    
    x=tf.keras.layers.Add()([x,x_skip])
    x=tf.keras.layers.Activation('relu')(x)
    
    return x


def ResNet152(shape=(256,256,3),classes=2):
    x_input=tf.keras.layers.Input(shape=shape)
    
    x=tf.keras.layers.Conv2D(64,kernel_size=7,strides=2,padding='same')(x_input)
    x=tf.keras.layers.BatchNormalization()(x)
    x=tf.keras.layers.Activation('relu')(x)
    x=tf.keras.layers.MaxPool2D(pool_size=3,strides=2,padding='same')(x)

    for i in range(3):
        x=residual_block(x,filters=64,downsample=(i==0))

    for i in range(8):
        x=residual_block(x,filters=128,stride=2 if i==0 else 1,downsample=(i==0))

    for i in range(36):
        x=residual_block(x,filters=256,stride=2 if i==0 else 1,downsample=(i==0))
    
    for i in range(3):
        x=residual_block(x,filters=512,stride=2 if i==0 else 1,downsample=(i==0))

    x=tf.keras.layers.GlobalAveragePooling2D()(x)

    y_outputs=tf.keras.layers.Dense(classes,activation='softmax')(x)

    model=tf.keras.models.Model(x_input,y_outputs)

    return model

In [3]:
model=ResNet152()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy","precision","recall"]
)

model.summary()

I0000 00:00:1741150450.372623    5802 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5563 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │      9,472 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128, 128,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ activation[0][0]  │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 64, 64,    │      4,160 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 64,    │     36,928 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 64,    │     16,640 │ activation_2[0][… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 64, 64,    │     16,640 │ max_pooling2d[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │      1,024 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 64, 64,    │          0 │ batch_normalizat… │
│                     │ 256)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 64, 64,    │          0 │ add[0][0]       

 Total params: 58,375,042 (222.68 MB)

 Trainable params: 58,223,618 (222.11 MB)

 Non-trainable params: 151,424 (591.50 KB)

In [4]:
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder

In [5]:
#Since there are no class labels in test.csv I have decided to do a classic 80/20 train-test split with the train.csv

BASE_DIR='./isic2020/isic2020'
TRAIN_CSV=os.path.join(BASE_DIR,'train.csv')
TEST_CSV=os.path.join(BASE_DIR,'test.csv')
TRAIN_DIR=os.path.join(BASE_DIR, 'train')
TEST_DIR=os.path.join(BASE_DIR,'test')

def get_train_image_path(image_name):
    path = os.path.join(TRAIN_DIR,image_name+'.jpg')
    if os.path.exists(path):
        return path
    print(f"Warning: Image {image_name} not found in any directory!")  # Debugging
    return None 

def get_test_image_path(image_name):
    path = os.path.join(TEST_DIR,image_name+'.jpg')
    if os.path.exists(path):
        return path
    print(f"Warning: Image {image_name} not found in any directory!")  # Debugging
    return None 

train_df=pd.read_csv(TRAIN_CSV)
test_df=pd.read_csv(TEST_CSV)

train_df['image_path']=train_df['image_name'].apply(get_train_image_path)
train_df=train_df.dropna(subset=["image_path"])
train_labels=train_df["benign_malignant"].values
train_label_encoder=LabelEncoder()
train_labels=train_label_encoder.fit_transform(train_labels)
num_classes=len(train_label_encoder.classes_)
if num_classes>1:
    train_labels=tf.keras.utils.to_categorical(train_labels,num_classes)

test_df['image_path']=test_df['image_name'].apply(get_test_image_path)
test_df=test_df.dropna(subset=["image_path"])
test_labels=test_df["benign_malignant"].values
test_label_encoder=LabelEncoder()
test_labels=test_label_encoder.fit_transform(test_labels)
num_classes=len(test_label_encoder.classes_)
if num_classes>1:
    test_labels=tf.keras.utils.to_categorical(test_labels,num_classes)

display(train_df)
display(test_df)

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,image_path
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0,./isic2020/isic2020/train/ISIC_2637011.jpg
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0,./isic2020/isic2020/train/ISIC_0015719.jpg
2,ISIC_0052212,IP_2842074,female,50.0,lower extremity,nevus,benign,0,./isic2020/isic2020/train/ISIC_0052212.jpg
3,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0,./isic2020/isic2020/train/ISIC_0068279.jpg
4,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0,./isic2020/isic2020/train/ISIC_0074268.jpg
...,...,...,...,...,...,...,...,...,...
30514,ISIC_9999127,IP_9583707,male,20.0,torso,unknown,benign,0,./isic2020/isic2020/train/ISIC_9999127.jpg
30515,ISIC_9999134,IP_6526534,male,50.0,torso,unknown,benign,0,./isic2020/isic2020/train/ISIC_9999134.jpg
30516,ISIC_9999320,IP_3650745,male,65.0,torso,unknown,benign,0,./isic2020/isic2020/train/ISIC_9999320.jpg
30517,ISIC_9999666,IP_7702038,male,50.0,lower extremity,unknown,benign,0,./isic2020/isic2020/train/ISIC_9999666.jpg


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target,image_path
0,ISIC_0077472,IP_3691360,female,40.0,torso,unknown,benign,0,./isic2020/isic2020/test/ISIC_0077472.jpg
1,ISIC_0080752,IP_2613684,male,50.0,torso,unknown,benign,0,./isic2020/isic2020/test/ISIC_0080752.jpg
2,ISIC_0084395,IP_0175539,female,45.0,torso,nevus,benign,0,./isic2020/isic2020/test/ISIC_0084395.jpg
3,ISIC_0086709,IP_4109313,male,30.0,torso,unknown,benign,0,./isic2020/isic2020/test/ISIC_0086709.jpg
4,ISIC_0098784,IP_1587853,female,70.0,lower extremity,unknown,benign,0,./isic2020/isic2020/test/ISIC_0098784.jpg
...,...,...,...,...,...,...,...,...,...
2177,ISIC_9992950,IP_4525077,female,65.0,upper extremity,unknown,benign,0,./isic2020/isic2020/test/ISIC_9992950.jpg
2178,ISIC_9998152,IP_4356379,female,40.0,lower extremity,unknown,benign,0,./isic2020/isic2020/test/ISIC_9998152.jpg
2179,ISIC_9998682,IP_2516168,male,60.0,head/neck,melanoma,malignant,1,./isic2020/isic2020/test/ISIC_9998682.jpg
2180,ISIC_9998965,IP_3293337,male,50.0,torso,unknown,benign,0,./isic2020/isic2020/test/ISIC_9998965.jpg


In [6]:
def load_image(image_path,label):
    image=tf.io.read_file(image_path)
    image=tf.image.decode_jpeg(image,channels=3)
    image=tf.image.resize(image,(256,256))
    image=image/255.0
    return image,label

train_dataset=tf.data.Dataset.from_tensor_slices((train_df["image_path"].values,train_labels))
train_dataset=train_dataset.map(load_image,num_parallel_calls=tf.data.AUTOTUNE)
BATCH_SIZE=16
train_dataset=train_dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

test_dataset=tf.data.Dataset.from_tensor_slices((test_df["image_path"].values,test_labels))
test_dataset=test_dataset.map(load_image,num_parallel_calls=tf.data.AUTOTUNE)
BATCH_SIZE=16
test_dataset=test_dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print(len(train_dataset))
print(len(test_dataset))

1908
137


In [7]:
print(train_dataset)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float64, name=None))>


In [12]:
history = model.fit(train_dataset, epochs=10)

Epoch 1/10


I0000 00:00:1740079758.904216   48234 service.cc:148] XLA service 0x7f0830004890 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1740079758.916467   48234 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-02-20 19:29:22.894435: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1740079781.198681   48234 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-02-20 19:29:48.792887: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_42041', 92 bytes spill stores, 92 bytes spill loads

2025-02-20 19:30:12.011242: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 3.530164454s
Trying algorithm eng46{k2=5,k5=3,k14=4} for conv (f32[16,64,64,6

1908/1908 ━━━━━━━━━━━━━━━━━━━━ 840s 328ms/step - accuracy: 0.9786 - loss: 0.1385 - precision: 0.9786 - recall: 0.9786
Epoch 2/10
1908/1908 ━━━━━━━━━━━━━━━━━━━━ 548s 286ms/step - accuracy: 0.9819 - loss: 0.0957 - precision: 0.9819 - recall: 0.9819
Epoch 3/10
1908/1908 ━━━━━━━━━━━━━━━━━━━━ 543s 284ms/step - accuracy: 0.9829 - loss: 0.0905 - precision: 0.9829 - recall: 0.9829
Epoch 4/10
1908/1908 ━━━━━━━━━━━━━━━━━━━━ 543s 284ms/step - accuracy: 0.9834 - loss: 0.0840 - precision: 0.9834 - recall: 0.9834
Epoch 5/10
1908/1908 ━━━━━━━━━━━━━━━━━━━━ 540s 282ms/step - accuracy: 0.9832 - loss: 0.0873 - precision: 0.9832 - recall: 0.9832
Epoch 6/10
1908/1908 ━━━━━━━━━━━━━━━━━━━━ 547s 286ms/step - accuracy: 0.9833 - loss: 0.0822 - precision: 0.9833 - recall: 0.9833
Epoch 7/10
1908/1908 ━━━━━━━━━━━━━━━━━━━━ 563s 285ms/step - accuracy: 0.9836 - loss: 0.0787 - precision: 0.9836 - recall: 0.9836
Epoch 8/10
1908/1908 ━━━━━━━━━━━━━━━━━━━━ 540s 282ms/step - accuracy: 0.9836 - loss: 0.0823 - precision: 0.9

In [13]:
print(history.history)

{'accuracy': [0.9812575578689575, 0.9818146228790283, 0.9821094870567322, 0.9822733402252197, 0.9822405576705933, 0.9822733402252197, 0.9822733402252197, 0.9822733402252197, 0.9822733402252197, 0.9822078347206116], 'loss': [0.10926730930805206, 0.09161759167909622, 0.092374287545681, 0.08916130661964417, 0.08951428532600403, 0.08730804920196533, 0.08304375410079956, 0.08828800916671753, 0.08729206025600433, 0.08663473278284073], 'precision': [0.9812575578689575, 0.9818146228790283, 0.9821094870567322, 0.9822733402252197, 0.9822405576705933, 0.9822733402252197, 0.9822733402252197, 0.9822733402252197, 0.9822733402252197, 0.9822078347206116], 'recall': [0.9812575578689575, 0.9818146228790283, 0.9821094870567322, 0.9822733402252197, 0.9822405576705933, 0.9822733402252197, 0.9822733402252197, 0.9822733402252197, 0.9822733402252197, 0.9822078347206116]}


In [14]:
test_loss,test_acc,test_precision,test_recall = model.evaluate(test_dataset, batch_size=16, verbose=1)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")
print(f"Test Precision: {test_precision}")
print(f"Test Recall: {test_recall}")
print(f"F1 Score: {2*(test_precision*test_recall)/(test_precision+test_recall)}")

137/137 ━━━━━━━━━━━━━━━━━━━━ 70s 182ms/step - accuracy: 0.9837 - loss: 0.0840 - precision: 0.9837 - recall: 0.9837
Test Loss: 0.08927512168884277
Test Accuracy: 0.9816681742668152
Test Precision: 0.9816681742668152
Test Recall: 0.9816681742668152
F1 Score: 0.9816681742668152
